In [ ]:
# import libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load and process data

In [ ]:
# Load data using pandas.read_csv()

data = pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
# View partial data

data.head()

* The first column is the label, and we need to extract it.

In [ ]:
# View data dimensions

data.shape

In [ ]:
# Extract the label column and convert it to numpy array

labels = data.pop('label').to_numpy()

In [ ]:
labels

In [ ]:
# Looking at the data dimensions again, the label column has been extracted

data.shape

* We first convert the data to a numpy array and then convert the flattened image to a two-dimensional image. Its shape is (28,28).

In [ ]:
data = data.to_numpy().reshape(-1,28,28)

* Let's show some pictures

In [ ]:
plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(data[i])
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(labels[i])

 * We will use a LeNet-style neural network for training, which requires input dimensions (None,32,32,1). So we need to add a dimension at the end of the array. But the image is 28*28, and to convert it to 32*32, we just need to fill in two rows or two columns with zeros around the image.

In [ ]:
# Add dimension

data = np.expand_dims(data,axis=-1)
labels = np.expand_dims(labels,axis=-1)

In [ ]:
# You can see that there's one more dimension at the end

data.shape

In [ ]:
labels.shape

In [ ]:
# Fill the image with zeros around it

data = tf.pad(data,[[0,0],[2,2],[2,2],[0,0]])

In [ ]:
# You can see the shape change

data.shape

* Next, we split the training dataset, validition dataset and test dataset with the ratio of 8:1:1 using tf.split.

In [ ]:
num = data.shape[0] // 10
train_data, val_data, test_data = tf.split(data,[num*8, num, num])
train_label, val_label, test_label = tf.split(labels,[num*8, num, num])

In [ ]:
print('train:',train_data.shape,'-- label:',train_label.shape)
print('validition:',val_data.shape,'-- label:',val_label.shape)
print('test:',test_data.shape,'-- label:',test_label.shape)

In [ ]:
# Show some pictures. Everything is all right

plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(train_data[i])
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(train_label[i][0].numpy())

* Convert training dataset,validition dataset and test dataset to tf.data.Dataset. Then change the order of data, set batch_size to 128 

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_data,train_label)).shuffle(33600,seed=42).batch(128)
val_ds = tf.data.Dataset.from_tensor_slices((val_data,val_label)).shuffle(33600,seed=42).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((test_data,test_label)).shuffle(33600,seed=42).batch(128)

In [ ]:
# Use 'take' to view a batch of data

plt.figure(figsize=(10,10))

for image,label in train_ds.take(1):
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(image[i])
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(label[i][0].numpy())

* In fact, the image pixels we are training should be scaled to 0-1, which is just divided by 255, but I'm not going to do this in data processing. Simply add a Rescaling layer at the bottom of the model, and you don't have to manually scale the data when making predictions.

# Build model

In [ ]:
# The shape of the picture

image_shape = (32,32,1)

In [ ]:
# Use Monte-Carlo Dropout to reduce overfitting

class MCDropout(layers.Dropout):
    def call(self,inputs):
        return super().call(inputs,training=True)


# Use a LeNet-style neural network. Some parameters are adjusted

inputs = layers.Input(shape=image_shape)
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs) # normalized
x = layers.Conv2D(128,5,strides=1,activation='relu')(x)
x = layers.MaxPooling2D(2,strides=2)(x)
x = layers.Conv2D(128,5,strides=1,activation='relu')(x)
x = layers.MaxPooling2D(2,strides=2)(x)
x = layers.Conv2D(128,5,strides=1,activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(84,activation='relu')(x)
x = MCDropout(0.1)(x)
outputs = layers.Dense(10)(x)

model = keras.Model(inputs,outputs)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# View network architecture
model.summary()

In [ ]:
# Using EarlyStopping, end training when val_accuracy is not improved for 10 consecutive times
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',
                                    patience=10,restore_best_weights=True)

# Using ReduceLROnPlateau, the learning rate is reduced by half when val_accuracy is not improved for 5 consecutive times
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5,patience=5)

# training
history = model.fit(train_ds,batch_size=128,epochs=100,validation_data=val_ds,
                    callbacks=[early_stopping,lr_scheduler])

* Draw the learning curve

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# test model
model.evaluate(test_ds,verbose=2)

# Submit

In [ ]:
# load data

test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
# process data

test = test.to_numpy().reshape(-1,28,28)
test = np.expand_dims(test,axis=-1)
test = tf.pad(test,[[0,0],[2,2],[2,2],[0,0]])

test.shape

In [ ]:
sample_submission.shape

In [ ]:
# predict
result =  model.predict(test)

result.shape

In [ ]:
# get predict label
predict_label = np.argmax(result,axis=-1)

predict_label.shape

In [ ]:
# Show some prediction results

plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(test[i,...,0])
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(predict_label[i])

In [ ]:
sample_submission['Label'] = predict_label

In [ ]:
sample_submission.to_csv('submission.csv', index=False)